# Eclipse Viewer — Observer's Perspective
Simulates what an eclipse looks like from any location on Earth.  
Select an eclipse, set your latitude/longitude, and use the **time slider** to watch the Moon cross the Sun.

In [19]:
import json
import re
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Dropdown

%matplotlib inline

In [ ]:
# ============================================================
# Load eclipse data exported by EclipseData.ipynb
# ============================================================
with open("eclipse_data.json") as f:
    data = json.load(f)
#loading in the json file that contains the parsed eclipse data in order for the visualization functions to calculate position given our data points.#
eclipse_list = data["eclipse_list"]
print(f"Loaded {len(eclipse_list)} eclipses from: {data['source_url']}")
print(f"Date range: {eclipse_list[0]['date_raw']}  →  {eclipse_list[-1]['date_raw']}")

Loaded 224 eclipses from: https://eclipse.gsfc.nasa.gov/SEcat5/SE2001-2100.html
Date range: 2001 Jun 21  →  2100 Sep 04


In [ ]:
# ============================================================
# Import shared eclipse utilities
# ============================================================
from eclipse_utils import (
    Eclipse,
    parse_coord,
    haversine_km,
    overlap_fraction,
    viewer_offset,
    get_eclipse_params
)

print("✓ Eclipse class & geometry helpers loaded from eclipse_utils.")

✓ Geometry helpers loaded.


In [22]:
# ============================================================
# DRAWING FUNCTION — renders the observer's sky view
# ============================================================

def draw_eclipse_sky(eclipse, obs_lat, obs_lon, time_frac):
    """
    Render what the eclipse looks like from (obs_lat, obs_lon).

    time_frac : float in [-1, 1]
        -1 = Moon approaching from the right
         0 = mid-eclipse (closest approach)
        +1 = Moon departing to the left
    """
    ecl_lat, ecl_lon, mag, path_w, _ = get_eclipse_params(eclipse)

    sun_r  = 1.0
    moon_r = mag * sun_r
    offset = viewer_offset(obs_lat, obs_lon, eclipse)

    # Moon travels horizontally across the Sun
    travel = sun_r + moon_r + 0.5
    mx = -time_frac * travel       # right → left
    my = offset

    # Center-to-center distance & obscuration
    d   = np.sqrt(mx**2 + my**2)
    obs_frac = overlap_fraction(sun_r, moon_r, d)

    dist_km = haversine_km(obs_lat, obs_lon, ecl_lat, ecl_lon)
    ecl_type = eclipse.get("type", "?")

    # ---- figure ----
    fig, ax = plt.subplots(figsize=(7, 7))
    ax.set_xlim(-2.5, 2.5)
    ax.set_ylim(-2.5, 2.5)
    ax.set_aspect("equal")

    # Sky color: darken with obscuration
    bright = max(0.02, 0.12 - 0.10 * obs_frac)
    ax.set_facecolor((bright, bright, bright + 0.12))

    # Corona glow — visible when >90 % obscured
    if obs_frac > 0.90:
        alpha_base = (obs_frac - 0.90) / 0.10 * 0.45
        for i in range(10):
            r = sun_r + 0.05 * (i + 1)
            a = alpha_base * (1 - i / 10)
            ax.add_patch(plt.Circle((0, 0), r,
                         color="#FFE4B5", alpha=max(a, 0), lw=0))

    # Sun disk
    ax.add_patch(plt.Circle((0, 0), sun_r, color="#FFD700", zorder=1))

    # Subtle limb-darkening rings
    for i in range(4):
        ax.add_patch(plt.Circle((0, 0), sun_r * (1 - 0.05 * i),
                     fill=False, ec="#FFA500", alpha=0.08, lw=1, zorder=1))

    # Moon disk
    ax.add_patch(plt.Circle((mx, my), moon_r, color="#111111", zorder=2))
    ax.add_patch(plt.Circle((mx, my), moon_r,
                 fill=False, ec="#333333", lw=1.5, zorder=3))

    # Stars when sky is dark enough
    if obs_frac > 0.85:
        rng = np.random.RandomState(42)
        n_stars = int(30 * (obs_frac - 0.85) / 0.15)
        sx = rng.uniform(-2.4, 2.4, n_stars)
        sy = rng.uniform(-2.4, 2.4, n_stars)
        ax.scatter(sx, sy, s=1, color="white", alpha=0.6, zorder=0)

    # Title & info
    ax.set_title(
        f"Eclipse View:  {eclipse['date_raw']}  —  {ecl_type}\n"
        f"Observer: {obs_lat:.1f}°N, {obs_lon:.1f}°E   |   "
        f"{dist_km:,.0f} km from center   |   "
        f"Obscuration: {obs_frac:.1%}",
        color="white", fontsize=11, fontweight="bold", pad=14,
    )
    info = f"Magnitude: {mag}   |   Saros: {eclipse.get('saros','?')}"
    if path_w > 0:
        info += f"   |   Path width: {path_w:.0f} km"
    ax.text(0, -2.35, info, ha="center", color="#aaaaaa", fontsize=9)
    ax.axis("off")
    plt.tight_layout()
    plt.show()

print("✓ Drawing function loaded.")

✓ Drawing function loaded.


In [23]:
# ============================================================
# INTERACTIVE VIEWER
# ============================================================
# • Dropdown — pick any of the 224 eclipses
# • Time slider — animate the Moon crossing the Sun
# • Lat / Lon sliders — move your observer position
#
# Tip: set continuous_update=False so the plot only redraws
#      when you release the slider (much smoother).
# ============================================================

# Build dropdown options
eclipse_options = {}
for i, e in enumerate(eclipse_list):
    label = f"{e['date_raw']}  —  {e['type']}  (Mag: {e['magnitude']})"
    eclipse_options[label] = i

def _update(idx, time_frac, lat, lon):
    draw_eclipse_sky(eclipse_list[idx], lat, lon, time_frac)

interact(
    _update,
    idx=Dropdown(options=eclipse_options, value=0, description="Eclipse:"),
    time_frac=FloatSlider(
        min=-1.0, max=1.0, step=0.02, value=-1.0,
        description="Time:", continuous_update=False,
        style={"description_width": "60px"},
        layout={"width": "500px"},
    ),
    lat=FloatSlider(
        min=-90, max=90, step=0.5, value=0.0,
        description="Lat (°N):", continuous_update=False,
        style={"description_width": "70px"},
        layout={"width": "500px"},
    ),
    lon=FloatSlider(
        min=-180, max=180, step=0.5, value=0.0,
        description="Lon (°E):", continuous_update=False,
        style={"description_width": "70px"},
        layout={"width": "500px"},
    ),
);

interactive(children=(Dropdown(description='Eclipse:', options={'2001 Jun 21  —  Total  (Mag: 1.0495)': 0, '20…